In [8]:
import os
import pandas as pd
import plotly.express as px
import progressbar
import json
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
pd.set_option('display.max_colwidth', 20)
import gzip


In [16]:
path = r'../../../../src//data/LocalBusiness/MFile_cleaned_withLBname'
dataList = []

with gzip.open(path, 'r') as dataFile:
    for line in dataFile:
        lineData = json.loads(line.decode('utf-8'))
        dataList.append(lineData)
data = pd.DataFrame(dataList)

In [27]:
data["clusterNtables"].unique()

array([ 2,  3,  4,  5,  6, 11,  9,  7, 12,  8, 13, 14, 10, 15])

In [3]:
def remove_stopwords(token_vector, stopwords_list):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in stopwords_list])


def remove_punctuation(token_vector):
    return token_vector.apply(lambda token_list: [word for word in token_list if word not in string.punctuation])

def jaccard_similarity_score(original, translation):
    intersect = set(original).intersection(set(translation))
    union = set(original).union(set(translation))
    try:
        return len(intersect) / len(union)
    except ZeroDivisionError:
        return 0
def delete_empty_rows(df):
    rows = []
    for i, row in enumerate(df['name']):
        if type(row) != str:
            rows.append(i)
    df_clean = df.drop(index=rows)
    df_clean = df_clean.reset_index(drop=True)
    return df_clean

In [144]:
len(data)

40389

In [44]:
# short pre-cleaning step to remove non-string entities
data = delete_empty_rows(data)
data['name'] = data['name'].apply(lambda row: row.lower())
data['business_tokens'] = data['name'].apply(lambda row: word_tokenize(row))
#data['business_tokens'] = remove_stopwords(data['business_tokens'],stopwords.words())
data['business_tokens'] = remove_punctuation (data['business_tokens'])
data.head()

,row_id,name,address,page_url,addressregion,streetaddress,addresscountry,addresslocality,postalcode,table_id,...,E.164 format,telephoneNorm,indexValue,MatchingNumbers,distance,MatchingNumbers_New,cluster_id,clusterNtables,tableNclusters,business_tokens
0,5860,le méridien visconti rome,"{'addresslocality': 'Rome', 'postalcode': '00193', 'streetaddress': 'Via Federico Cesi 37', 'addresscountry': 'Italy'}",https://whattoexpect.marriott.com/rommd,None,VIA FEDERICO CESI 37,IT,ROME,00193,Hotel_marriott.com_September2020.json.gz,...,39063684,39063684,591585,"[591585, 589400, 587459]","[-1, 0.0, 0.0]","[591585, 589400, 587459]",1,2,3096,"[le, méridien, visconti, rome]"
1,5229,residence inn boston franklin,"{'addressregion': 'Massachusetts', 'postalcode': '02038', 'addresslocality': 'Franklin', 'streetaddress': '4 Forge Parkway', 'addresscountry': 'USA'}",https://www.marriott.com/hotels/local-things-to-do/bosfr-residence-inn-boston-franklin/,Massachusetts,4 FORGE PARKWAY,US,FRANKLIN,02038,Hotel_marriott.com_September2020.json.gz,...,15085418188,15085418188,591212,"[591212, 585732, 602392]","[-1, 0.0, 0.0]","[591212, 585732, 602392]",4342,3,3096,"[residence, inn, boston, franklin]"
2,5971,residence inn boston foxborough,"{'addresslocality': 'Foxborough', 'addressregion': 'Massachusetts', 'postalcode': '02035', 'streetaddress': '250 Foxborough Blvd', 'addresscountry': 'USA'}",https://www.marriott.com/hotels/travel/bosfx-residence-inn-boston-foxborough/,Massachusetts,250 FOXBOROUGH BLVD,US,FOXBOROUGH,02035,Hotel_marriott.com_September2020.json.gz,...,15086982800,15086982800,591654,"[587766, 591654, 600468]","[0.0, -1, 0.0]","[587766, 591654, 600468]",4359,3,3096,"[residence, inn, boston, foxborough]"
3,7383,residence inn boston bridgewater,"{'postalcode': '02324', 'addressregion': 'Massachusetts', 'addresslocality': 'Bridgewater', 'streetaddress': '2020 Pleasant Street', 'addresscountry': 'USA'}",https://www.marriott.com/hotels/hotel-reviews/ewbbw-residence-inn-boston-bridgewater/,Massachusetts,2020 PLEASANT STREET,US,BRIDGEWATER,02324,Hotel_marriott.com_September2020.json.gz,...,15086971412,15086971412,592477,"[592477, 601701]","[-1, 0.0]","[592477, 601701]",4358,2,3096,"[residence, inn, boston, bridgewater]"
4,5552,courtyard boston natick,"{'addressregion': 'Massachusetts', 'addresslocality': 'Natick', 'postalcode': '01760', 'streetaddress': '342 Speen Street', 'addresscountry': 'USA'}",https://www.marriott.com/hotels/hotel-deals/bosnf-courtyard-boston-natick/,Massachusetts,342 SPEEN STREET,US,NATICK,01760,Hotel_marriott.com_September2020.json.gz,...,15086556100,15086556100,591406,"[591406, 587365]","[-1, 0.0]","[591406, 587365]",4354,2,3096,"[courtyard, boston, natick]"


In [8]:
len(data)

40389

In [45]:
#get tagged words
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(data['business_tokens'])]
# build model and vocabulary
model = Doc2Vec(vector_size=50, min_count = 5, epochs = 25, dm = 0)
model.build_vocab(tagged_data)
# Train model
model.train(tagged_data, total_examples=model.corpus_count, epochs=25)

In [46]:
tagged_data[:10]

[TaggedDocument(words=['le', 'méridien', 'visconti', 'rome'], tags=['0']),
 TaggedDocument(words=['residence', 'inn', 'boston', 'franklin'], tags=['1']),
 TaggedDocument(words=['residence', 'inn', 'boston', 'foxborough'], tags=['2']),
 TaggedDocument(words=['residence', 'inn', 'boston', 'bridgewater'], tags=['3']),
 TaggedDocument(words=['courtyard', 'boston', 'natick'], tags=['4']),
 TaggedDocument(words=['residence', 'inn', 'boston', 'natick'], tags=['5']),
 TaggedDocument(words=['courtyard', 'boston', 'milford'], tags=['6']),
 TaggedDocument(words=['courtyard', 'boston', 'milford'], tags=['7']),
 TaggedDocument(words=['residence', 'inn', 'boston', 'brockton/easton'], tags=['8']),
 TaggedDocument(words=['courtyard', 'boston', 'foxborough/mansfield'], tags=['9'])]

## Start from top clusters - seed clusters

In [154]:
# isolate top clusters with at least 14 tables
df_14 = data[data["clusterNtables"] >= 14]
# save top cluster ids in list
top_clusters_list = df_14['cluster_id'].unique().tolist()
print("Number of individual entities:" , len(top_clusters_list))
print("Number of tables:" , df_14["table_id"].nunique())

Number of individual entities: 26
Number of tables: 30


In [155]:
df_14["tableNclusters"].sort_values()

39601       4
38785       8
38602       9
37981      13
37584      15
         ... 
15302    1234
15326    1234
15338    1234
14830    1234
14132    1234
Name: tableNclusters, Length: 376, dtype: int64

In [152]:
df_14.loc[39601]["table_id"]

'LocalBusiness_visitnordfyn.com_September2020.json.gz'

### **Question**: Is that a Problem? Is there a min_size for the tables?

# Now try to expand from seed-clusters and get 4 similar entities

- idea: given we have x seed clusters, we identify 4 similar entities (=cluster_ids) and have 4*x individual final entities
- note the similar entities should exhibit a cluster size of at least 10 (15)

In [160]:
index_top_clusters_list=[]
for i_d in top_clusters_list:
    index_top_clusters_list.append(data[data['cluster_id']==i_d].index[0])


In [191]:
index_top_clusters_list[:10]

[14132, 14268, 14286, 14313, 14437, 14522, 14614, 14618, 14730, 14756]

In [173]:
model.docvecs.most_similar(14522, topn = 20)

C:\Users\Jannik\AppData\Local\Temp/ipykernel_20740/1030062797.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  model.docvecs.most_similar(14522, topn = 20)


[('21129', 0.989883303642273),
 ('23176', 0.9890414476394653),
 ('19122', 0.9870344996452332),
 ('25435', 0.9867845773696899),
 ('22569', 0.9864322543144226),
 ('16801', 0.9860634207725525),
 ('25233', 0.9859685301780701),
 ('19008', 0.985758364200592),
 ('20586', 0.9856730103492737),
 ('23802', 0.9856458306312561),
 ('16996', 0.9855813384056091),
 ('20580', 0.9848409295082092),
 ('23609', 0.98475581407547),
 ('25373', 0.9846353530883789),
 ('18786', 0.9845947027206421),
 ('20414', 0.9845492839813232),
 ('15926', 0.9840709567070007),
 ('19555', 0.9838147163391113),
 ('35520', 0.983170747756958),
 ('19991', 0.9828994870185852)]

In [113]:
tagged_data[14522]

TaggedDocument(words=['euro', 'mobil', 'autohouse'], tags=['14522'])

In [174]:
tagged_data[19122]

TaggedDocument(words=['euro', 'mobil', 'autohouse'], tags=['19122'])

In [186]:
tagged_data[20580]

TaggedDocument(words=['euro', 'mechanik'], tags=['20580'])

In [189]:
data.iloc[int(14522)]["cluster_id"]

2053

In [181]:
data.iloc[int(22569)]["cluster_id"]

2053

In [190]:
data.iloc[int(20580)]["cluster_id"]

7220

### Find similar entities based on Jaccard and Doc2Vec 

- similarity metric ``name``

In [197]:
clusters_search=[]
for i in index_top_clusters_list:
    similar_doc = model.docvecs.most_similar(i, topn = 25)
    clusters_search.append(i)
    for index, similarity in similar_doc:
        if data.iloc[int(index)]['cluster_id'] != data.iloc[int(i)]['cluster_id']:
            if data.iloc[int(index)]['clusterNtables']>10:
                clusters_search.append(int(index))
    jaccard_score = data['business_tokens'].apply(lambda row: jaccard_similarity_score(row,data.iloc[i]['business_tokens']))
    indizes=sorted(range(len(jaccard_score)), key=lambda i: jaccard_score[i])[-20:]
    for index in indizes:
        if data.iloc[int(index)]['cluster_id'] != data.iloc[int(i)]['cluster_id']:
             if data.iloc[int(index)]['clusterNtables']>10:
                clusters_search.append(int(index))    
df_final = data.iloc[clusters_search]

C:\Users\Jannik\AppData\Local\Temp/ipykernel_20740/3956135039.py:3: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(i, topn = 25)


In [198]:
len(df_final)

161

In [208]:
print("Total number of cluster_ids:" , df_final["cluster_id"].nunique())
print("Total number of tables:" , df_final["table_id"].nunique())

Total number of cluster_ids: 69
Total number of tables: 17


In [217]:
df_final.groupby("table_id")["tableNclusters"].unique().sort_values()

table_id
LocalBusiness_igotbiz.com_September2020.json.gz              [58]
LocalBusiness_subieshops.com_September2020.json.gz           [72]
LocalBusiness_visitdenmark.com_September2020.json.gz        [206]
LocalBusiness_exoticautoshops.com_September2020.json.gz     [259]
LocalBusiness_infinitishops.com_September2020.json.gz       [259]
LocalBusiness_acushops.com_September2020.json.gz            [308]
LocalBusiness_saabshops.com_September2020.json.gz           [332]
LocalBusiness_lexrepairshops.com_September2020.json.gz      [442]
LocalBusiness_jagshops.com_September2020.json.gz            [461]
LocalBusiness_lrshops.com_September2020.json.gz             [523]
LocalBusiness_volvomechanics.com_September2020.json.gz      [577]
LocalBusiness_pcarshops.com_September2020.json.gz           [778]
LocalBusiness_vcarshops.com_September2020.json.gz           [833]
LocalBusiness_minirepairshops.com_September2020.json.gz     [842]
LocalBusiness_benzshops.com_September2020.json.gz           [901]
L

df_final might still contain **duplicate cluster_ids** due top 20 similarity scores**


In [219]:
df_final_clean = df_final.drop_duplicates('cluster_id', keep='first')
len(df_final_clean)

69

In [220]:
df_final_clean["cluster_id"].nunique()

69

In [221]:
#df_final_clean.to_excel("LB_selection.xlsx")

- Which attributes are involved for matching?
- weighted Jaccard
- min size of table?
- what if do not get along with cluster_size = 15?